In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def setSelenium():
    options=Options()
    options.add_argument('start-maximized')
    options.add_argument('--headless')
    driver=webdriver.Firefox(options=options,executable_path=r'Y:\geckodriver.exe')
    return driver

In [3]:
def getHTML(website,selDriver=None,reqDriver=False,reqParam={},reqHeader={}):
    if selDriver !=None:
        fetch = selDriver.get(website)
        rtrnPage = BeautifulSoup(selDriver.page_source,'html.parser')
        selDriver.quit()
        return rtrnPage.prettify()
    if reqDriver:
        with requests.Session() as fetch:
            itemFetch = fetch.get(website,params=reqParam,headers=reqHeader)
            if itemFetch.status_code == 200:
                return itemFetch.json()
            else:
                return 'Problem with request, request code [{statusCode}]'.format(statusCode=itemFetch.status_code)
    return 

In [4]:
city_url = 'https://raw.githubusercontent.com/ndejong/philippines-data/master/data/2015%20Population%20Density_web_clean-columns.json'

In [5]:
def setCityData(path_url,local_available=False):
    if local_available:
        return pd.read_csv(path_url)
    fetch_res = getHTML(website=path_url,reqDriver=True)
    cityData = {i:list() for i in fetch_res[0].keys()}
    for i in fetch_res:
        for x in i.keys():
            cityData[x].append(i[x])
    city_data = pd.DataFrame.from_dict(cityData)
    return city_data

In [6]:
setCityData(path_url=city_url)

,region,region_name,province,city_municipality,is_capital,population_2015,land_area_sqkm,population_density
0,NCR,National Capital Region,NCR,City of Manila,no,1780148,24.98,71263
1,NCR,National Capital Region,NCR,Quezon City,no,2936116,171.71,17099
2,NCR,National Capital Region,NCR,Caloocan City,no,1583978,55.80,28387
3,NCR,National Capital Region,NCR,Pasay City,no,416522,13.97,29815
4,NCR,National Capital Region,NCR,City of Las Piñas,no,588894,32.69,18014
...,...,...,...,...,...,...,...,...
1629,ARMM,Autonomous Region in Muslim Mindanao,Tawi-Tawi,Simunul,no,31223,167.25,187
1630,ARMM,Autonomous Region in Muslim Mindanao,Tawi-Tawi,Sitangkai,no,33334,735.46,45
1631,ARMM,Autonomous Region in Muslim Mindanao,Tawi-Tawi,South Ubian,no,25935,272.04,95
1632,ARMM,Autonomous Region in Muslim Mindanao,Tawi-Tawi,Tandubas,no,29390,552.05,53


In [7]:
def goto_mainLocPage(main_url,city_search,driver=None):
    data_return ={
        #returns url, parsed_html, formatted_html
    }
    search_url = '/Search?q='
    city_add = city_search if len(city_search.split()) <2 else city_search.split().replace(' ','+')
    fixed_url = main_url+search_url+city_add
    ownDriver = driver if driver != None else setSelenium()
    ownDriver = setSelenium()
    ownDriver.get(fixed_url)
    data_return['url'] = str(ownDriver.current_url)
    data_return['parsed_html'] = BeautifulSoup(ownDriver.page_source,'html.parser')
    data_return['formatted_html'] = data_return['parsed_html'].prettify()
    ownDriver.quit()
    return data_return

In [8]:
main_url='https://www.tripadvisor.com.ph'

In [10]:
data_return = goto_mainLocPage(main_url=main_url,city_search='cavite',driver=None)

In [11]:
def fetch_TourPages(data_return):
    page_s = []
    divs_find = data_return['parsed_html'].find_all('div',attrs={'class':'ui_columns is-mobile result-content-columns'})
    for i in divs_find:
        if str(i.attrs['onclick'].split(',')[3][3:10]) == 'Tourism':
            page_s.append(str(i.attrs['onclick'].split(',')[3][3:-1]))
    return page_s

In [12]:
tourist_pages = fetch_TourPages(data_return = data_return)
tourist_pages

['Tourism-g6090426-Cavite_Province_Calabarzon_Region_Luzon-Vacations.html',
 'Tourism-g298448-Cavite_City_Cavite_Province_Calabarzon_Region_Luzon-Vacations.html']

In [36]:
def fetch_subpages(main_url,which_section,pages,driver=None):
    #should return a list of url's
    pages_init = []
    subpages_final = [] #final batch of url's
    ownDriver = driver if driver != None else setSelenium()
    subpages_pages = [] #pages of each subpages
    # Hotels, Restaurants, ShowForum
    for i in pages: #should return or store main hotel-city link/s 
        exp_page = main_url + '/' + i
        ownDriver.get(exp_page)
        fetchPageNums = BeautifulSoup(ownDriver.page_source,'html.parser')
        for html in fetchPageNums.find_all('a',attrs={'class':'ebMYO Ra _S z _Z w o v _Y Wh k _T ddFHE'}):
            ref = str(html.attrs['href'])
            try:
                if ref[ref.index('/')+1:ref.index('-')] == which_section:
                    pages_init.append(main_url + ref)
            except ValueError:
                continue
        
    for i,y in enumerate(pages_init): #should return all hotel pages
        ownDriver.get(y)
        fetchPageNums = BeautifulSoup(ownDriver.page_source,'html.parser')
        fetchedNums = fetchPageNums.find_all('div',attrs={'class':'pageNumbers'})
        for a in fetchedNums:
            for b in a.find_all('a'):
                subpages_pages.append(main_url+str(b.attrs['href']))
          
    for a in subpages_pages:
        ownDriver.get(a)
        fetchPageSource = BeautifulSoup(ownDriver.page_source,'html.parser')
        fetchHotels = fetchPageSource.find_all('a',attrs={'class':'property_title prominent'})
        for i in fetchHotels:
            subpages_final.append(main_url + str(i.attrs['href']))
    ownDriver.quit()
    ownDriver.quit()
    ownDriver.quit()
    return subpages_final if len(subpages_final) else 'Source Non-Exsistent'
        

In [37]:
fetch_business = fetch_subpages(main_url=main_url,which_section='Hotels',pages=tourist_pages,driver=None)

https://www.tripadvisor.com.ph/Hotels-g6090426-Cavite_Province_Calabarzon_Region_Luzon-Hotels.html
https://www.tripadvisor.com.ph/Hotels-g298448-Cavite_City_Cavite_Province_Calabarzon_Region_Luzon-Hotels.html


In [38]:
fetch_business

['https://www.tripadvisor.com.ph/Hotel_Review-g317121-d1605241-Reviews-The_Lake_Hotel_Tagaytay-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html',
 'https://www.tripadvisor.com.ph/Hotel_Review-g317121-d320846-Reviews-Taal_Vista_Hotel-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html',
 'https://www.tripadvisor.com.ph/Hotel_Review-g317121-d1605241-Reviews-The_Lake_Hotel_Tagaytay-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html',
 'https://www.tripadvisor.com.ph/Hotel_Review-g317121-d8317906-Reviews-Hotel_Monticello-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html',
 'https://www.tripadvisor.com.ph/Hotel_Review-g317121-d1046421-Reviews-Hotel_Kimberly_Tagaytay-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html',
 'https://www.tripadvisor.com.ph/Hotel_Review-g317121-d15246382-Reviews-Quest_Hotel-Tagaytay_Cavite_Province_Calabarzon_Region_Luzon.html',
 'https://www.tripadvisor.com.ph/Hotel_Review-g317121-d320846-Reviews-Taal_Vista_Hotel-Tagaytay_Cavite_Province_Calabarz

In [9]:
fetch_web = 'https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Manila_Luzon.html'

In [10]:
def fetch_avePrice(web_url,driver=None):
    ownDriver = driver if driver != None else setSelenium()
    ownDriver.get(web_url)
    fetch = BeautifulSoup(ownDriver.page_source,'html.parser')
    fet = fetch.find_all('div', attrs={'id':'taplc_resp_hr_atf_meta_component_0'})
    nums =[]
    for i in fet:
        textVer = i.text.split()
        for i in textVer:
            try:
                if '₱' not in i:
                    continue
                else:
                    value = ''
                    for x in i:
                        if x.isnumeric() == True:
                            value += x
                        if (x.isalpha() == True) or x == '₱':
                            nums.append(value)
                            value = ''
            except ValueError:
                continue
    final_nums = [int(i) for i in nums if i.isnumeric()==True ]
    ownDriver.quit()
    return '{0:.2f}'.format(np.mean(final_nums)) if len(final_nums) != 0 else 'NA'

In [11]:
def separate_string(word=''):
    words_list = []
    switch = 0
    for i,letters in enumerate(word):
        if letters in ['/',' ','(',')','-',"\'"]:
            continue
        if letters.isnumeric() == True:
            if word[i-1].isnumeric():
                continue
            else:
                words_list.append(word[switch:i-1])
        if letters.upper() == letters:
            if (word[i-1].upper() == word[i-1]):
                continue
            words_list.append(word[switch:i])
            switch = i
    return words_list

In [12]:
def fetchAmenities(fetch_var):
    if fetch_var == None:
        return {}
    if len(fetch_var) != 0:
        features = ['hotel_amenities','room_features','room_types','extra']
        pull_quality = {features[i]:list() for i in range(len(fetch_var))}
        for y,i in enumerate(fetch_var):
            for x in i:
                if len(str(x.text)) > 50:
                    for b in separate_string(str(x.text)):
                        pull_quality[features[y]].append(b)
                else:
                    pull_quality[features[y]].append(str(x.text))
        return pull_quality
    else:
        return {}

In [13]:
def fetchTraveler_rating(fetch_var):
    travel_ratings = {}
    if (fetch_var != None) or (len(fetch_var) != 0):
        for rate,rating in zip(fetch_var[0].find_all('label',attrs={'class':'Oixff Vm _S fxJOd'}),fetch_var[0].find_all('span',attrs={'class':'cpqJw'})):
            travel_ratings['rating_'+rate.text.lower()] = rating.text.lower()
            
        for lang,no_lang in zip(fetch_var[-1].find_all('span',attrs={'class':'fwSIg q'}),fetch_var[-1].find_all('span',attrs={'class':'cvxmR'})):
            travel_ratings['lang_'+lang.text.lower()] = no_lang.text[1:-1]
    else:
        return {}
    return travel_ratings

In [14]:
def fetchCommon_words(fetch_var):
    if (fetch_var == None) or (len(fetch_var) == 0):
        return {}
    words_common = {'words_list':list()}
    for i in fetch_var:
        words_common['words_list'].append(str(i.text))
    words_common['no_common'] = len(words_common['words_list'])
    return words_common

In [17]:
def getHotelData(website,driver=None):
    #return no reviews, address, name, ranking
    ownDriver = driver if driver!= None else setSelenium()
    ownDriver.get(website)
    page_url = ownDriver.current_url
    pageFetch = BeautifulSoup(ownDriver.page_source,'html.parser')
    # create dictionary for looping
    
    hotel_rank = str(pageFetch.find('b',attrs={'class','rank'}).text)[1:] if pageFetch.find('b',attrs={'class','rank'}) != None else 'NA'
    review_no = str(pageFetch.find('span',attrs={'class','HFUqL'}).text).split()[0] if pageFetch.find('span',attrs={'class','HFUqL'}) != None else 'NA'
    is_covid_safe = 'YES' if len(pageFetch.find_all('div',attrs={'class':'ddXRv'})) != 0 else 'NO'
    hotel_address = str(pageFetch.find('span',attrs={'class','ceIOZ yYjkv'}).text) if pageFetch.find('span',attrs={'class','ceIOZ yYjkv'}) != None else 'NA'
    hotel_name = str(pageFetch.find('h1',attrs={'id','fkWsC b d Pn'}).text) if pageFetch.find('h1',attrs={'id','fkWsC b d Pn'}) != None else 'NA'
    hotel_grade = str(pageFetch.find('span',attrs={'class','bvcwU P'}).text) if pageFetch.find('span',attrs={'class','bvcwU P'}) != None else 'NA'
    hotel_price = float(fetch_avePrice(web_url = website,driver=None))
    walk = pageFetch.find('span',attrs={'class':'bpwqy dfNPK'}).text
    resto = pageFetch.find('span',attrs={'class':'bpwqy VyMdE'}).text
    spots = pageFetch.find('span',attrs={'class':'bpwqy eKwbS'}).text
    hotel_data = {
        'hotel_url':page_url,
        'review_no': review_no,
        'hotel_address': hotel_address,
        'hotel_name': hotel_name,
        'hotel_rank': hotel_rank,
        'covSafe_inclusion': is_covid_safe,
        'hotel_grade': hotel_grade,
        'hotel_ave_price':hotel_price,
        'hotel_location_grade':'NA',
        'hotel_cleanliness_grade':'NA',
        'hotel_service_grade':'NA',
        'hotel_value_grade':'NA',
        'no_hotel_amenities':'NA',
        'no_room_features':'NA',
        'no_room_types':'NA',
        'location_walkable_score':walk,
        'location_resto_count':resto,
        'location_attraction_spots':spots
        
    }
    
    room_qual = fetchAmenities(fetch_var=pageFetch.find_all('div',attrs={'class':'exmBD K'}))
    if (len(room_qual.keys()) != 0) or room_qual != None:
        for i in room_qual.keys():
            hotel_data['no_'+i] = len(room_qual[i])

    for i in pageFetch.find_all('div',attrs={'class':'cmZRz f'}):
        hotel_data['hotel_'+str(i.text).lower() + '_grade'] = i.find('span').attrs['class'][1][-2:] if i.find('span').attrs['class'][1][-2:] != None else 'NA'
    hotel_data['hotel_value_grade'] = pageFetch.find('div',attrs={'class':'cmZRz f dfnfs'}).find('span').attrs['class'][1][-2:] if pageFetch.find('div',attrs={'class':'cmZRz f dfnfs'}) != None else 'NA' 
    
    traveler = pageFetch.find_all('ul',attrs={'class':'mPill w S4'})
    traveler_rates = fetchTraveler_rating(fetch_var=traveler)
    for i in traveler_rates.keys():
        hotel_data['traveler_{data}'.format(data=i)]=int(traveler_rates[i])            
    
    words_uni = pageFetch.find('div',attrs={'class':'cqLsE'})
    hotel_data['common_words_review']=fetchCommon_words(fetch_var=words_uni)['words_list']
    
    
    
    ownDriver.quit()
    return hotel_data
    
    

In [19]:
hotel_dataAriel = getHotelData(website=fetch_web,driver=None)

In [28]:
test = fetch_web
test_name = 'Joy Nostalg Hotel Suites Manila Managed by AccorHotels'

In [33]:
def getHotelReviews(main_url,hotel_url,hotel_name,driver=None):
    hotel_main = hotel_url[:-4] + '#REVIEW'
    hotel_pages = [hotel_main]
    review_props = ['rev_date','rev_title','rev_type','rev_rating','rev_par']
    review_data ={i:'NA' for i in review_props}
    rev_data_full = []
    ownDriver = driver if driver != None else setSelenium()
    ownDriver.get(hotel_main)
    page_fetched = BeautifulSoup(ownDriver.page_source,'html.parser')
    ownDriver.quit()
    
    for a in page_fetched.find('div',attrs={'class':'pageNumbers'}).find_all('a'):
        hotel_pages.append(main_url + a.attrs['href'])
        
    if len(hotel_pages) > 1:
        while True:
            initial= hotel_pages[-1]
            print('digging---'+initial)
            initial2 = initial[initial.index('-or'):initial.index(hotel_name.replace(' ','_'))]
            new_page = int(initial2[3:-1])+5
            page_active = initial.replace(initial2,'-or'+str(new_page)+'-')
            driver12 = driver if driver != None else setSelenium()
            driver12.get(page_active)
            if driver12.current_url == page_active:
                hotel_pages.append(page_active)
            else:
                driver12.quit()
                break
            driver12.quit()
    for b in hotel_pages:
        print('extracting----'+b)
        driver2 = driver if driver!= None else setSelenium()
        driver2.get(b)
        fetched2 = BeautifulSoup(driver2.page_source,'html.parser')
        driver2.quit()
        for i in fetched2.find_all('div',attrs={'class':'cWwQK MC R2 Gi z Z BB dXjiy'}):
            rev_date = i.find('span',attrs={'class':'euPKI _R Me S4 H3'}).text.replace('Date of stay: ','') if i.find('span',attrs={'class':'euPKI _R Me S4 H3'}) != None else 'NA'
            rev_rating = i.find('div',attrs={'class':'emWez F1'}).find('span').attrs['class'][1][-2:] if i.find('div',attrs={'class':'emWez F1'}) != None else 'NA'
            rev_title = i.find('a',attrs={'class':'fCitC'}).find('span').text if i.find('a',attrs={'class':'fCitC'}).find('span') != None else 'NA'
            rev_type = i.find('span',attrs={'class':'eHSjO _R Me'}).text[i.find('span',attrs={'class':'eHSjO _R Me'}).text.index(':')+1:] + '\n' if i.find('span',attrs={'class':'eHSjO _R Me'}) != None else 'NA'
            rev_par = i.find('q',attrs={'class':'XllAv H4 _a'}).find('span').text if i.find('q',attrs={'class':'XllAv H4 _a'})!= None else 'NA'
            
            review_data['rev_date'] = rev_date
            review_data['rev_rating']= rev_rating
            review_data['rev_title']= rev_title
            review_data['rev_type']= rev_type
            review_data['rev_par']=rev_par
            
            rev_data_full.append(review_data)
            review_data ={i:'NA' for i in review_props}
            time.sleep(5)
    return rev_data_full


In [34]:
hotel_rev = getHotelReviews(main_url = 'https://www.tripadvisor.com.ph',hotel_url=test,hotel_name=test_name,driver=None) 

digging---https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-or25-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Manila_Luzon.html
digging---https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-or30-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Manila_Luzon.html
digging---https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-or35-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Manila_Luzon.html
digging---https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-or40-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Manila_Luzon.html
digging---https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-or45-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Manila_Luzon.html
digging---https://www.tripadvisor.com.ph/Hotel_Review-g298453-d1484778-Reviews-or50-Joy_Nostalg_Hotel_Suites_Manila_Managed_by_AccorHotels-Pasig_Metro_Mani

In [37]:
hotel_review = pd.DataFrame.from_dict(hotel_rev)

In [44]:
hotel_review.to_csv('ariel_hotelReviews.csv')

In [42]:
data = [{'date':'now'},{'date':'how'}]
with open('arielHotel_metadata.json','a+') as js:
    json.dump(hotel_dataAriel,js)

In [79]:
with open('testDump.json','r+') as js:
    data2 = json.load(js)

In [80]:
pd.DataFrame.from_dict(data2)

,date
0,now
1,how


In [ ]:
'''
code struct::
1st loop -- store pages html (add the first page which is html+#REVIEW)
2nd loop(nested and stored into a previously initialized dictionary)
---1 = pages html
---2 = insert for loop above

dict = {title, date, type, rating, paragraph review}

1.) store every page html
2.) if there are none then just focus on the first page
3.) configure the for loop above 
'''